In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
# Hyper Parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 64
TIME_STEP = 28          # rnn time step / image height
INPUT_SIZE = 28         # rnn input size / image width
LR = 0.01               # learning rate
DOWNLOAD_MNIST = True   # set to True if haven't download the data

# Mnist digital dataset
train_data = dsets.MNIST(
    root='/home/hecong/dplearn/data/mnist/',
    train=True,                         # this is training data
    transform=transforms.ToTensor(),    # Converts a PIL.Image or numpy.ndarray to
                                        # torch.FloatTensor of shape (C x H x W) and normalize in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,            # download it if you don't have it
)


In [4]:
# Data Loader for easy mini-batch return in training
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# convert test data into Variable, pick 2000 samples to speed up testing
test_data = dsets.MNIST(root='/home/hecong/dplearn/data/mnist/', train=False, transform=transforms.ToTensor())
test_x = Variable(test_data.test_data, volatile=True).type(torch.FloatTensor)[:2000]/255.   # shape (2000, 28, 28) value in range(0,1)
test_y = test_data.test_labels.numpy().squeeze()[:2000]    # covert to numpy array


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [5]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=INPUT_SIZE,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 10)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out


rnn = RNN()
print(rnn)

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted


RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
from stock.common.log import Logger
logger = Logger('/home/hecong/dplearn/base/log')

# training and testing
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):        # gives batch data
        b_x = Variable(x.view(-1, 28, 28))              # reshape x to (batch, time_step, input_size)
        b_y = Variable(y)                               # batch y

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        if step % 50 == 0:
            test_output = rnn(test_x)                   # (samples, time_step, input_size)
            pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
            accuracy = sum(pred_y == test_y) / float(test_y.size)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)
            logger.scalar_summary('loss',loss.data[0],step)
            logger.scalar_summary('accuracy',accuracy,step)
            for name, param in rnn.named_parameters():
                logger.histo_summary('layer_wb/' + name, param.clone().cpu().data.numpy(), step)
                if param.grad is not None:
                    logger.histo_summary('layer_gd/' + name+'/grad', param.grad.clone().cpu().data.numpy(), step)
            


/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/hecong/venv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch:  0 | train loss: 2.3173 | test accuracy: 0.10
Epoch:  0 | train loss: 0.9930 | test accuracy: 0.56
Epoch:  0 | train loss: 0.8853 | test accuracy: 0.68
Epoch:  0 | train loss: 0.5808 | test accuracy: 0.77
Epoch:  0 | train loss: 0.4360 | test accuracy: 0.82
Epoch:  0 | train loss: 0.4570 | test accuracy: 0.87
Epoch:  0 | train loss: 0.2564 | test accuracy: 0.90
Epoch:  0 | train loss: 0.1771 | test accuracy: 0.92
Epoch:  0 | train loss: 0.2051 | test accuracy: 0.92
Epoch:  0 | train loss: 0.3090 | test accuracy: 0.92
Epoch:  0 | train loss: 0.3545 | test accuracy: 0.94
Epoch:  0 | train loss: 0.2967 | test accuracy: 0.93
Epoch:  0 | train loss: 0.2119 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1583 | test accuracy: 0.94
Epoch:  0 | train loss: 0.2194 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1759 | test accuracy: 0.94
Epoch:  0 | train loss: 0.0942 | test accuracy: 0.94
Epoch:  0 | train loss: 0.1303 | test accuracy: 0.95
Epoch:  0 | train loss: 0.2961 | test accuracy